# Práctica 9

In [3]:
import scipy.stats
import numpy as np
import math

- <a href="#Ejercicio-1">Ejercicio 1</a>
- <a href="#Ejercicio-2">Ejercicio 2</a>
- <a href="#Ejercicio-3">Ejercicio 3</a>
- <a href="#Ejercicio-4">Ejercicio 4</a>
- <a href="#Ejercicio-5">Ejercicio 5</a>

## Ejercicio 1

Un productor agropecuario cultiva una variedad de soja cuyo rendimiento por hectárea
(ha.) puede considerarse una variable aleatoria con distribución N(37; 25). Un vendedor
de granos pone a la venta una nueva variedad de soja, diciendo que tiene un
rendimiento mayor. El productor agropecuario realizó una compra de esta nueva variedad
y quiere asegurarse de que tienen un rendimiento mayor que las semillas que
compra habitualmente. Con ese fin decide cultivar 10 parcelas de 1 ha., observar sus
rendimientos y aplicar un test de hipótesis de nivel 0.05. Suponiendo que el rendimiento
por ha. de la nueva variedad de soja tiene una distribución normal con varianza 25,
resolver los siguientes ítems.

### Pregunta A

Definir las variables aleatorias y los parámetros involucrados en este problema y
plantear el test de hipótesis que debe realizar el productor agropecuario.

* $A$ : "Rendimiento de la i-ésima hectarea plantada con la variedad inicial."
* $B$ : "Rendimiento de la i-ésima hectarea plantada con la nueva variedad."
* $B_{i}, ... , B_{10}$ variables aleatorias con distribución $V_{2,i} \sim N(37;25)$
* $E(A) = 37$
* $Var(A) = 25$
* $\alpha = 0.05$
* $Var(B) = 25$

Defino el test de hipótesis como:

* $H_0$ : "La nueva variedad no mejora el rendimiento" $\Rightarrow \mu_0 = 37$
* $H_a$ : "La nueva variedad mejora el rendimiento" $\Rightarrow \mu > \mu_0$

Y el pivote, dado que los datos son normales:

$T = \frac{\overline{X_n} - \mu_0}{\sigma/\sqrt{n}} \sim N(0,1)$

### Pregunta B

El productor cultiva las 10 parcelas y obtiene los siguientes rendimientos:
37 - 39.50 - 41.70 - 42 - 40 - 41.25 - 43 - 44.05 - 38 - 38.50
Cuál es la conclusión del test en base a estos datos observados?

In [4]:
datos = [37, 39.50, 41.70, 42, 40, 41.25, 43, 44.05, 38, 38.50]

In [28]:
def rechazarH0NormalesParaMuConVar(tipo, alfa, mediaMuestral, mu_0, sigma, n):
    """
    Tipo 0: mu < mu_0
    Tipo 1: mu > mu_0
    Tipo 2: mu != mu_0
    """
    va = scipy.stats.norm()
    k = (mediaMuestral - mu_0)/(sigma/math.sqrt(n))
    print("mu = {}".format(k))
    if tipo == 0:
        z = -va.ppf(1-alfa)
        print("z = {}".format(z))
        if k < z:
            return True
        else: return False
    elif tipo == 1:
        z = va.ppf(1-alfa)
        print("z = {}".format(z))
        if k > z:
            return True
        else: return False
    else:
        z = va.ppf(1-alfa/2)
        print("z = {}".format(z))
        if (k < -z) or (k > z):
            return True
        else: return False

In [29]:
tipo = 1
alfa = 0.05
mediaMuestral = np.mean(datos)
mu = 37
sigma = math.sqrt(25)
n = 10
rechazarH0NormalesParaMuConVar(tipo, alfa, mediaMuestral, mu, sigma, n)

mu = 2.2135943621178655
z = 1.6448536269514722


True

**Luego se decide rechazar la hipótesis nula con probabilidad de error de tipo I igual a 5%**

Calcular la
probabilidad del error de tipo II suponiendo que el verdadero rendimiento medio
de la nueva variedad de soja es de 40 toneladas por ha.

Dado que el verdadero $\mu$ es distinto a $37$ como habíamos considerado con la hipótesis nula, debemos rearmar el pivote con $\mu = 40$

$\beta(\mu) = \phi\left(z_{1-\alpha} + \frac{\mu_0 - \mu}{\sigma/\sqrt{n}}\right) = \phi\left(z_{1-0.05} + \frac{37 - 40}{5/\sqrt{10}}\right) = \phi\left(z_{0.95} + \frac{-3}{5/\sqrt{10}}\right)$

In [7]:
def probaErrorTipoII(tipo, alfa, mu_0, mu, sigma, n):
    """
    Tipo 0: mu < mu_0
    Tipo 1: mu > mu_0
    Tipo 2: mu != mu_0
    """
    va = scipy.stats.norm()
    const = (mu_0 - mu)/(sigma/math.sqrt(n))
    if tipo == 0:
        z = -va.ppf(1-alfa)
        return 1 - va.cdf(z+const)
    elif tipo == 1:
        z = va.ppf(1-alfa)
        return va.cdf(z+const)
    else:
        z = va.ppf(1-alfa/2)
        return va.cdf(z+const) - va.cdf(-z+const)

In [8]:
tipo = 1
alfa = 0.05
mu_0 = 37
mu = 40
sigma = 5
n = 10
probaErrorTipoII(tipo, alfa, mu_0, mu, sigma, n)

0.400322295453562

**Luego la probabilidad de error de tipo II es del 40%**

### Pregunta D

Determinar el número n de parcelas a cultivar para que la probabilidad del error de tipo II sea menor o igual que 0.05 suponiendo que el verdadero rendimiento medio de la nueva variedad de soja es de 40 toneladas por ha.

$
\begin{align*}
\beta(\mu) < \epsilon &\iff \phi\left(z_{1-\alpha} + \frac{\mu_0 - \mu}{\sigma}\sqrt{n}\right) < \epsilon\\
&\iff z_{1-\alpha} + \frac{\mu_0 - \mu}{\sigma}\sqrt{n} < z_{\epsilon}\\
&\iff n > \left(\frac{(z_{\epsilon}-z_{1-\alpha})\sigma}{\mu-\mu_0}\right)^2\\
\end{align*}$

In [11]:
def nParaErrotTipoIIMenorEpsilon(tipo, alfa, epsilon, sigma, mu_0, mu):
    """
    Tipo 0: mu < mu_0
    Tipo 1: mu > mu_0
    Tipo 2: mu != mu_0
    """
    va = scipy.stats.norm()
    if tipo == 0:
        z = -va.ppf(1-alfa)
        e = va.ppf(1-epsilon)
        return (((e+z)*sigma)/(mu_0-mu))**2
    elif tipo == 1:
        z = va.ppf(1-alfa)
        e = va.ppf(epsilon)
        return (((e-z)*sigma)/(mu-mu_0))**2
    else:
        return "No hay n"

In [12]:
tipo = 1
alfa = 0.05
epsilon = 0.05
sigma = 5
mu_0 = 37
mu = 40
nParaErrotTipoIIMenorEpsilon(tipo, alfa, epsilon, sigma, mu_0, mu)

30.06159393439349

**Luego se deben cultivar 30 parcelas para que la probabilidad de error de tipo II sea menor o igual al 0.05**

### Pregunta E

Calcular la función de potencia $\pi(\mu)$ del test planteado en c), verificar que es
creciente para $\mu \in R$ y deducir que este test sirve tambi en para testear

$H_0 : \mu \leq 37\\$
$H_1 : \mu > 37$


In [ ]:
def funcionPotencia(mu, mu_0):
    if mu == mu_0:
        return pro

## Ejercicio 2

En la construcción de un edificio debe usarse un concreto que tenga una tensión media mayor a 300 psi. Para verificar si el concreto preparado a partir del cemento "Loma Blanca" cumple con este requerimiento, se realizan 15 mediciones en forma independiente de la tensión de este concreto. Se observa una media muestral de 304 psi y un desvío estándar muestral de 10 psi. El constructor est a dispuesto a correr un riesgo del 5% de comprar cemento "Loma Blanca" cuando este produce un concreto que no cumple con las especificaciones. Suponiendo que los datos están normalmente distribuidos:

### Pregunta A

Plantear el test correspondiente. ¿Qué decisión se toma?

Datos
* $\mu = 300$
* $n = 15$
* $\overline{X_n} = 304$
* $sd = 10$
* $\alpha = 0.05$

Test de hipótesis

* $H_0 \Rightarrow \mu_0 = 300$
* $H_a \Rightarrow \mu > \mu_0$

Dado que tenemos el desvío estándar muestral, hay que cambiar el pivote por la T de student.

In [26]:
def rechazarH0NormalesParaMuSinVar(tipo, alfa, mediaMuestral, mu_0, desvioMuestral, n):
    """
    Tipo 0: mu < mu_0
    Tipo 1: mu > mu_0
    Tipo 2: mu != mu_0
    """
    va = scipy.stats.t(df=n-1)
    k = (mediaMuestral - mu_0)/(desvioMuestral/math.sqrt(n))
    print("mu = {}".format(k))
    if tipo == 0:
        z = -va.ppf(1-alfa)
        print("z = {}".format(z))
        if k < z:
            return True
        else: return False
    elif tipo == 1:
        z = va.ppf(1-alfa)
        print("z = {}".format(z))
        if k > z:
            return True
        else: return False
    else:
        z = va.ppf(1-alfa/2)
        print("z = {}".format(z))
        if (k < -z) or (k > z):
            return True
        else: return False

In [27]:
tipo = 1
alfa = 0.05
mediaMuestral = 304
mu = 300
desvioMuestral = 10
n = 10
rechazarH0NormalesParaMuSinVar(tipo, alfa, mediaMuestral, mu, desvioMuestral, n)

mu = 1.2649110640673518
z = 1.8331129326536335


False

**Luego no se rechaza la hipótesis nula**

### Pregunta B

Acotar el $p$-valor.

El p-valor es la probabilidad límite con la que se acepta la hipótesis nula.

In [21]:
va = scipy.stats.t(df=n-1)
mu = 1.2649110640673518
1-va.cdf(mu)

0.11883628158949289

**Luego el p-value es 11.88%**

In [24]:
#Verifico que a partir de este valor se rechaza la H0
tipo = 1
alfa = 0.11883628158949289 # p-value
mediaMuestral = 304
mu = 300
desvioMuestral = 10
n = 10
rechazarH0NormalesSinVar(tipo, alfa, mediaMuestral, mu, desvioMuestral, n)

mu = 1.2649110640673518
z = 1.2649110640672758


True

## Ejercicio 3

Se diseñó un nuevo sistema de riego de manera tal que el desvío del tiempo de activación sea menor que 6 segundos. Se lo prueba 11 veces, obteniéndose los siguientes tiempos de activación:

27 41 22 27 23 35 30 24 27 28 22

Suponiendo que el tiempo de activación (en segundos) es una v.a. con distribución normal:

### Pregunta A

¿Usted decidiría, a nivel 0.05, que el sistema cumple la especificación?

Test de hipótesis para la varianza de normales:

$H_0 \Rightarrow \sigma_0^2 = 36\\$
$H_a \Rightarrow \sigma^2 < \sigma_0^2$

Definición del pivote

$T = \frac{\sum{\left(xi-\overline{X_n}\right)^2}}{\sigma_0^2} = \frac{(n-1)s^2}{\sigma_0^2} \sim \chi_{n-1}^2 $

In [44]:
def rechazarH0NormalesParaVarSinMu(tipo, alfa, sigma_0, desvioMuestral, n):
    """
    Tipo 0: mu < mu_0
    Tipo 1: mu > mu_0
    Tipo 2: mu != mu_0
    """
    va = scipy.stats.chi2(df=n-1)
    k = ((n-1)*(desvioMuestral**2))/(sigma_0**2)
    print("var = {}".format(k))
    if tipo == 0:
        z = va.ppf(alfa)
        print("z = {}".format(z))
        if k < z:
            return True
        else: return False
    elif tipo == 1:
        z = va.ppf(1-alfa)
        print("z = {}".format(z))
        if k > z:
            return True
        else: return False
    else:
        z0 = va.ppf(alfa/2)
        z1 = va.ppf(1-alfa/2)
        print("z = {}".format((z0,z1)))
        if (k < z0) or (k > z1):
            return True
        else: return False

In [45]:
datos = [27, 41, 22, 27, 23,35, 30, 24, 27, 28, 22]

In [46]:
tipo = 0
alfa = 0.05
sigma_0 = 6
desvioMuestral = np.std(datos,ddof=1)
n = 11
rechazarH0NormalesParaVarSinMu(tipo, alfa, sigma_0, desvioMuestral, n)

var = 9.378787878787879
z = 3.9402991361190605


False

**Luego no se rechaza la hipótesis nula**

### Pregunta B
Acotar el $p$-valor

In [51]:
va = scipy.stats.chi2(df=n-1)
sigma = 9.378787878787879
va.cdf(sigma)

0.5034283426853524

In [53]:
#Verifico que a partir de este valor se rechaza la H0
tipo = 0
alfa = 0.5034283426853525
sigma_0 = 6
desvioMuestral = np.std(datos,ddof=1)
n = 11
rechazarH0NormalesParaVarSinMu(tipo, alfa, sigma_0, desvioMuestral, n)

var = 9.378787878787879
z = 9.37878787878788


True

**Luego el p-value es igual a 50.34%**

## Ejercicio 4

Se supone que 1 de cada 10 fumadores prefiere la marca A. Después de una campaña publicitaria en cierta región de ventas, se entrevistó a 200 fumadores para determinar la efectividad de la campaña. El resultado de esta encuesta mostró que 26 personas preferían la marca A.

### Pregunta A

¿Indican estos datos, a nivel aproximado 0.05, un aumento en la preferencia por la marca A?

Datos
* $p = \frac{1}{10}$
* $n = 200$
* $\alpha = 0.05$
* $\overline{X_n} = \frac{26}{200}$

Es un experimento de varaibles Bernoulli

Test de hipótesis

$H_0 \Rightarrow p_0 = 0.1\\$
$H_a \Rightarrow p > p_0$

Luego defino el pivote como

$T = \frac{\overline{X_n}-p_0}{\sqrt{p_0\cdot(1-p_0)}}\sqrt{n} \sim N(0,1)$

In [55]:
def rechazarH0Bernoulli(tipo, alfa, mediaMuestral, p_0, n):
    """
    Tipo 0: p < p_0
    Tipo 1: p > p_0
    Tipo 2: p != p_0
    """
    va = scipy.stats.norm()
    k = (mediaMuestral - p_0)/(math.sqrt(p_0*(1-p_0))/math.sqrt(n))
    print("p = {}".format(k))
    if tipo == 0:
        z = -va.ppf(1-alfa)
        print("z = {}".format(z))
        if k < z:
            return True
        else: return False
    elif tipo == 1:
        z = va.ppf(1-alfa)
        print("z = {}".format(z))
        if k > z:
            return True
        else: return False
    else:
        z = va.ppf(1-alfa/2)
        print("z = {}".format(z))
        if (k < -z) or (k > z):
            return True
        else: return False

In [56]:
tipo = 1
alfa = 0.05
mediaMuestral = 26/200
p_0 = 0.1
n = 200
rechazarH0Bernoulli(tipo, alfa, mediaMuestral, p_0, n)

p = 1.414213562373095
z = 1.6448536269514722


False

**Luego no rechazo la hipótesis nula y por lo tanto no hay un aumento en la preferencia de A**

### Pregunta B

Calcular el $p$-valor aproximado.

In [58]:
va = scipy.stats.norm()
p = 1.414213562373095
1-va.cdf(p)

0.07864960352514261

In [59]:
# Verifico que se rechaza la H0 para ese valor de alfa
tipo = 1
alfa = 0.07864960352514261
mediaMuestral = 26/200
p_0 = 0.1
n = 200
rechazarH0Bernoulli(tipo, alfa, mediaMuestral, p_0, n)

p = 1.414213562373095
z = 1.4142135623730945


True

### Pregunta C

¿Cuál es la probabilidad aproximada de decidir que la campaña publicitaria no fue efectiva, cuando en realidad la proporción de preferencia por la marca A después de la campaña es 0.15?

Pide calcular la probabilidad de cometer error del tipo II

In [61]:
tipo = 1
alfa = 0.05
p_0 = 0.1
p = 0.15
sigma = math.sqrt(p*(1-p))
n = 200
probaErrorTipoII(tipo, alfa, p_0, p, sigma, n)

0.36864604896345954

**Luego hay una proba de 36,86% de no rechazar la $H_0$ cuando esta es falsa**

### Pregunta E

¿Qué tamaño de muestra debería tomarse para que la probabilidad de c) fuese a lo sumo 0.05?

In [62]:
tipo = 1
alfa = 0.05
epsilon = 0.05
sigma = math.sqrt(p*(1-p))
p_0 = 0.1
p = 0.15
nParaErrotTipoIIMenorEpsilon(tipo, alfa, epsilon, sigma, p_0, p)

551.9308646354644

In [65]:
# Verifico la proba de error de tipo II con lo obtenido antes
tipo = 1
alfa = 0.05
p_0 = 0.1
p = 0.15
sigma = math.sqrt(p*(1-p))
n = 551.9308646354644
probaErrorTipoII(tipo, alfa, p_0, p, sigma, n)

0.050000000000000044

**Luego debería tomarse una muestra de por lo menos 552 personas**

## Ejercicio 5

*****REVISAR*****

Se supone que el tiempo de duración de cierto tipo de lamparitas tiene distribución exponencial. Una fábrica garantiza que el tiempo medio de vida de las lamparitas que produce es mayor que 50 días, y la empresa vendedora quiere asegurarse que la producción satisface las especificaciones antes de sacarla a la venta. Para ello toma al azar una muestra de 40 lamparitas y observa el tiempo de duración de las mismas, obteniendo un promedio de 53 días.

### Pregunta A

Proponer un test de nivel exacto con hipótesis nula simple para este problema de manera tal que la probabilidad de que la empresa decida no vender si no se satisfacen los requerimientos sea del 0.95.

Datos
* $E(x_i) = \frac{1}{\lambda} = 50 \Rightarrow \lambda = \frac{1}{50}$
* $X_i \sim Exp(\frac{1}{50})$
* $n = 40$
* $\overline{X_n} = 53$

Test de hipótesis

$H_0 \Rightarrow \mu_0 = 50\\$
$H_a \Rightarrow \mu > \mu_0$

Usando lo visto en intervalos de confianza para exponencial <a href="https://campus.exactas.uba.ar/pluginfile.php/304400/course/section/38573/PyECcompleta.pdf#page=208">Apunte Bianco</a>

El intervalo de confianza del parámetro $\lambda$ queda definido como:

$ IC = \left[\frac{\chi_{2n, 1-\alpha/2}^2}{2\sum{X_i}}, \frac{\chi_{2n, \alpha/2}^2}{2\sum{X_i}}\right]$

In [76]:
# Revisar
def rechazarH0ExponExacto(tipo, alfa, mediaMuestral, mu_0, desvioMuestral, n):
    """
    Tipo 0: mu < mu_0
    Tipo 1: mu > mu_0
    Tipo 2: mu != mu_0
    """
    va = scipy.stats.chi2(df=2*n)
    k = 1/mu_0
    print("var = {}".format(k))
    if tipo == 0:
        z = va.ppf(1-alfa/2)
        print("z = {}".format(z))
        if k < z:
            return True
        else: return False
    elif tipo == 1:
        z = va.ppf(alfa/2)
        print("z = {}".format(z))
        if k > z:
            return True
        else: return False
    else:
        z0 = va.ppf(1-alfa/2)
        z1 = va.ppf(alfa/2)
        print("z = {}".format((z0,z1)))
        if (k < z0) or (k > z1):
            return True
        else: return False

In [77]:
tipo = 1
alfa = 0.05
mediaMuestral = 53
mu_0 = 50
desvioMuestral = 1/53
n = 40
rechazarH0ExponExacto(tipo, alfa, mediaMuestral, mu_0, desvioMuestral, n)

var = 1005.6042959335448
z = 101.87947396543588


True

### Pregunta C

Repetir a) usando un nivel aproximado.

Plantear el pivote usando el TCL:

$T = \frac{\overline{X_n} - \mu_0}{\sigma/\sqrt{n}} \sim N(0,1)$

In [81]:
def rechazarH0ExponAprox(tipo, alfa, mediaMuestral, mu_0, sigma, n):
    """
    Tipo 0: mu < mu_0
    Tipo 1: mu > mu_0
    Tipo 2: mu != mu_0
    """
    va = scipy.stats.norm()
    k = (mediaMuestral - mu_0)/(sigma/math.sqrt(n))
    print("mu = {}".format(k))
    if tipo == 0:
        z = -va.ppf(1-alfa)
        print("z = {}".format(z))
        if k < z:
            return True
        else: return False
    elif tipo == 1:
        z = va.ppf(1-alfa)
        print("z = {}".format(z))
        if k > z:
            return True
        else: return False
    else:
        z = va.ppf(1-alfa/2)
        print("z = {}".format(z))
        if (k < -z) or (k > z):
            return True
        else: return False

In [83]:
tipo = 1
alfa = 0.05
mediaMuestral = 53
mu = 50
sigma = 50
n = 40
rechazarH0ExponAprox(tipo, alfa, mediaMuestral, mu, sigma, n)

mu = 0.37947331922020555
z = 1.6448536269514722


False

**Luego no rechazo la hipótesis nula**

### Pregunta E

Utilizando el test aproximado, ¿qué probabilidad tiene la empresa de no sacar la producción a la venta, si el promedio de vida verdadero es 52 días?

Pide la probabilidad de cometer error de tipo II

In [84]:
tipo = 1
alfa = 0.05
mu_0 = 50
mu = 52
sigma = 50
n = 40
probaErrorTipoII(tipo, alfa, mu_0, mu, sigma, n)

0.9180193302239402

**Luego hay un 91,80% de probabilidad de no sacar a producción si el promedio de vida es de 52 días**

### Pregunta F

¿Cuál debería ser el tamaño de la muestra para que la probabilidad de d) fuese 0.1?

In [85]:
tipo = 1
alfa = 0.05
epsilon = 0.1
sigma = 50
mu_0 = 50
mu = 52
nParaErrotTipoIIMenorEpsilon(tipo, alfa, epsilon, sigma, p_0, p)

8563847.350667976